Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [1]:
import numpy as np
import cv2

from ipynb.fs.full.fonctions import display
# Repeatability
np.random.seed(0)

VFILENAME = "video_record/1310.mp4"
HEIGHT = 406
WIDTH = 722

c:\Users\lc100\AppData\Local\miniconda3\envs\frameSubtraction\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\lc100\AppData\Local\miniconda3\envs\frameSubtraction\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\lc100\AppData\Local\miniconda3\envs\frameSubtraction\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"



very blurry
(32, 32, 3)
(64, 64, 3)
28
28
29
28
---------------------------------------------------------------------------------------------
[[[28 60 69]
  [29 60 69]]

 [[28 60 69]
  [28 63 69]]] 
------
 [[[30 62 71]
  [30 62 71]]

 [[30 74 59]
  [30 62 71]]]

------
 14.0
---------------------------------------------------------------------------------------------

blurry
(32, 32, 3)
(32, 32, 3)
28
28
29
28
---------------------------------------------------------------------------------------------
[[[28 60 69]
  [29 60 69]]

 [[28 60 69]
  [28 63 69]]] 
------
 [[[30 54 84]
  [30 53 85]]

 [[30 54 84]
  [30 53 85]]]

------
 32.666666666666664
---------------------------------------------------------------------------------------------

hand
(32, 32, 3)
(32, 32, 3)
28
28
29
28
---------------------------------------------------------------------------------------------
[[[28 60 69]
  [29 60 69]]

 [[28 60 69]
  [28 63 69]]] 
------
 [[[  0   0 127]
  [  0   0 126]]

 [[ 14   5 1

Load video frames from file

In [2]:
def get_frames(filename):
    video=cv2.VideoCapture(filename)
    #print(np.shape(video))
    while video.isOpened():
        ret,frame=video.read()
        if ret:
            yield frame
        else:
            break
    video.release()
    yield None

Creating a particle cloud

In [3]:
NUM_PARTICLES=200
VEL_RANGE=0.5

def initialize_particles(N=NUM_PARTICLES,velocity=VEL_RANGE,width=WIDTH,height=HEIGHT):
    particles=np.random.rand(N,4)
    particles=particles*np.array((height,width,velocity,velocity))
    particles[:,2:4] -= velocity/2.0
    #print(particles[:20,:])
    return particles

Moving particles according to their velocity state

In [4]:
def apply_velocity(particles):
    particles[:,0] += particles[:,2]
    particles[:,1] += particles[:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [5]:
def enforce_edges(particles,N=NUM_PARTICLES,width=WIDTH,height=HEIGHT):
    for i in range(N):
        
        #print("     before: ",particles[i,0],particles[i,1],particles[i,2],particles[i,3])
        vitesse=40
        
        if particles[i,0]>height-1:
            particles[i,0]=height-1-vitesse
            particles[i,2]=-vitesse
        if particles[i,1]> width-1:
            particles[i,1]=width-1-vitesse
            particles[i,3]=-vitesse
            
        if particles[i,0]< 0:
            particles[i,0]=vitesse
            particles[i,2]=vitesse
        if particles[i,1]< 0:
            particles[i,1]=vitesse
            particles[i,3]=vitesse
            
        #print("     after: ",particles[i,0],particles[i,1])
            
    return particles

Measure each particle's quality

In [6]:
TARGET_COLOUR = np.array((156,74,38))
TARGET_COLOUR_WHITE = np.array((45,40,205))
DIFF_COLOR=(150,100,100)

def compute_errors(particles, frame,N=NUM_PARTICLES,colour=TARGET_COLOUR_WHITE, diff=DIFF_COLOR, coeffW=255*3):
    color_lower=colour-diff
    errors=np.zeros(N)
    for i in range(N):
        y=int(particles[i,0])
        x=int(particles[i,1])
        
        #print("     pixel: ",x,y)
        
        pixel_colour=frame[y,x,:]
        
        GB=abs(pixel_colour[1]-pixel_colour[0])
        GR=abs(pixel_colour[1]-pixel_colour[2])
        
        if all(color_lower<pixel_colour) and GB<coeffW and GR<coeffW:
            errors[i]=np.sum((colour-pixel_colour)**2)
            #print(color_lower,pixel_colour,errors[i])
        else:
            errors[i]=10000000
            
        #errors[i]=np.sum((colour-pixel_colour)**2)
        
    return errors

Assign weights to the particles based on their quality of match

In [7]:
def compute_weights(errors,particles,width=WIDTH,height=HEIGHT):
    weights=np.max(errors)-errors
    weights[
        (particles[:,0]==0) |
        (particles[:,0] == height-1) |
        (particles[:,1]==0) |
        (particles[:,1] == width-1) 
    ]=0.0
    weights=weights**4
    return weights

Resample particles according to their weights

In [8]:
def resample(particles, weights,N=NUM_PARTICLES):
    weights+=1.e-100
    somme=np.sum(weights)
    
    probabilities = weights/somme
    index_numbers=np.random.choice(
        N,
        size=N,
        p=probabilities
    )
    
    particles=particles[index_numbers,:]
    y=np.mean(particles[:,0])
    x=np.mean(particles[:,1])
    return particles, (int(x), int(y))

Fuzz the particles

In [9]:
POS_SIGMA = 1.0
VEL_SIGMA = 0.5
def apply_noise(particles,N=NUM_PARTICLES,sigma=POS_SIGMA):
    noise=np.concatenate(
        (
        np.random.normal(0.0, sigma, (N,1)),
        np.random.normal(0.0, sigma, (N,1)),
        np.random.normal(0.0, sigma, (N,1)),
        np.random.normal(0.0, sigma, (N,1)),
    ),
    axis=1)
    particles+=noise
    return particles

Display the video frames

In [10]:
def display_particle(frame, particles, location,N=NUM_PARTICLES):
    if len(particles) > 0:
        for i in range(N):
            y=int(particles[i,0])
            x=int(particles[i,1])
            cv2.circle(frame,(x,y),1,(0,255,0),5)
    if len(location) > 0:
        cv2.circle(frame, (location[0],location[1]), 15,(0,0,255),5)
    
    display(frame,name='particle')
    
    if cv2.waitKey(30)==27:
        if cv2.waitKey(0)==27:
            return True
            
    return False

Main routine

In [11]:
TARGET_COLOUR = np.array((156,74,38))
TARGET_COLOUR_WHITE = np.array((200,200,200))
POS_SIGMA = 1.0
VEL_SIGMA = 0.5
NUM_PARTICLES=200
VEL_RANGE=500
DIFF_COLOR=(150,100,100)

WIDTH=1920
HEIGHT=1080

def particlesDetect(particles,frame,N=NUM_PARTICLES,width=WIDTH,height=HEIGHT,colour=TARGET_COLOUR_WHITE,sigma=POS_SIGMA,diff_color=DIFF_COLOR, coeffW_error=255*3):
    
    particles = apply_velocity(particles)
    particles = enforce_edges(particles,N=N,width=width,height=height)
    errors = compute_errors(particles, frame, N=N,colour=colour,diff=diff_color,coeffW=coeffW_error)
    weights = compute_weights(errors,particles,width=width,height=height)
    particles, location = resample(particles, weights,N=N)
    particles = apply_noise(particles,N=N,sigma=sigma)
    terminate = display_particle(frame, particles, location,N=N)
    
    return particles,terminate,location

In [12]:
"""
W=1920
H=1080
print(W,H)
N=10
POS_SIGMA = 10.0
VEL=50.0
particles = initialize_particles(N=N,width=W,height=H,velocity=VEL)

TARGET_COLOUR_WHITE = np.array((200,200,200))
for frame in get_frames(VFILENAME):
    #print(frame.shape)
    #print(frame[1080-1,1920-1,: ])
    if frame is None: break

    particles,terminate=particlesDetect(particles,frame,N=N,width=W,height=H,sigma=POS_SIGMA)
    
    if terminate:break
"""

'\nW=1920\nH=1080\nprint(W,H)\nN=10\nPOS_SIGMA = 10.0\nVEL=50.0\nparticles = initialize_particles(N=N,width=W,height=H,velocity=VEL)\n\nTARGET_COLOUR_WHITE = np.array((200,200,200))\nfor frame in get_frames(VFILENAME):\n    #print(frame.shape)\n    #print(frame[1080-1,1920-1,: ])\n    if frame is None: break\n\n    particles,terminate=particlesDetect(particles,frame,N=N,width=W,height=H,sigma=POS_SIGMA)\n    \n    if terminate:break\n'